In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from utils import *
from training_text_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())




C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
# Chargement des données
import load_data
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=False)

In [3]:
data[275:285]   

,ipu_id,speaker,start,stop,text,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,dyad
275,275,OR,428.38,429.915,enfin tu sais qu'on a pas le rattrapage ça fai...,True,False,False,NaN,False,False,transcr\AAOR
276,276,AA,429.20,429.375,ouais,False,False,False,NaN,False,False,transcr\AAOR
277,277,OR,431.02,432.415,non c'est juin la période non,True,False,False,NaN,False,False,transcr\AAOR
278,278,AA,431.04,431.235,mh,False,False,False,NaN,False,False,transcr\AAOR
279,279,OR,433.18,434.315,début juin je crois même hein,True,False,True,433.960,False,False,transcr\AAOR
280,280,AA,433.92,434.295,mai,True,True,False,NaN,False,False,transcr\AAOR
281,281,AA,434.52,434.735,non,True,False,True,434.793,False,True,transcr\AAOR
282,282,OR,435.72,436.860,ah même pour les tu penses,True,True,False,NaN,False,False,transcr\AAOR
283,283,AA,436.10,437.425,c'était en mai non je crois les partiels,False,False,False,NaN,False,False,transcr\AAOR
284,284,OR,437.42,441.935,moi je croyais qu'on les avait toujours eu en ...,True,False,True,441.935,False,False,transcr\AAOR


# creation des y à prédire

In [4]:

df = pd.DataFrame(data)

print("y de base")
y=create_y(df) # Création de la target
print(y.count(1))
print(y.count(0))


print("y yield at")
y=create_y_yield_at(df) # Création de la target
print(y.count(1))
print(y.count(0))

print("y turn after")
y=create_y_turn_after(df) # Création de la target
print(y.count(1))
print(y.count(0))
print(len(y))
print(len(data))

y de base
9131
7269
y yield at
3090
13310
y turn after
3030
13370
16400
16400


In [5]:



##features = data['text'].to_numpy().reshape(-1).tolist() 


# remplacement des valeurs nulles par [UNK] pour le tokenizer
#features = [word if not pd.isnull(word) else '[UNK]' for word in features]


from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Fixez le seed pour la reproductibilité
seed = 42

data_text=create_sequences(data)
# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data_text, y, test_size=0.2, random_state=seed)




# Vérification des dimensions
print(f"Nombre de séquences d'entraînement: {len(X_train)}, étiquettes: {len(y_train)}")
print(f"Nombre de séquences de test: {len(X_test)}, étiquettes: {len(y_test)}")


Nombre de séquences d'entraînement: 13120, étiquettes: 13120
Nombre de séquences de test: 3280, étiquettes: 3280


In [6]:
print(X_train)
print(y_train[365:375])

['l e   u n', "m a i s   t u   s a i s   q u e   ç a   m ' e s t   a r r i v é   a u s s i   à   q u a n d   j e   b o s s a i s   c h e z   m a n g o   m a i s", "o u i   o u i   c ' e s t   v r a i   j e   l ' a v a i s   d é j à   v u   e n   p l u s   d e d a n s", 'c o m m e n t', "à   p a r t i r   d e   l ' a n n é e   o ù   j ' a i   c o m m e n c é   à   p a s s e r   l e   b a c   j ' a v a i s   u n   u n   v r a i   p e t i t   b o u l o t", 'j e   r e s t e r a i   a v e c   t o i   u n e   s e m a i n e', 'p o u r   a v o i r', "b a h   j e   t r o u v e   q u ' o n   a v a i t   v r a i m e n t   p a s   b e a u c o u p   d e   d e   c h o i x   l à", "j -   v e u x   y   a r r i v e r   e t   t o u t   m a i s   e u h   o n   m ' e n   o n   m e   l ' a   d i t   h e i n   o n   m ' a   d i t   l a   L 2   c ' e s t   p a s", 'm h   m h', 'u n k', "o u a i s   ç a   c ' e s t   c ' e s t   c h a c u n   c ' e s t   c h a c u n", "u n e   l a r g e   f o u l e   s ' e s 

In [7]:
# Paramètres
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer pour CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)

# Création des datasets et dataloaders
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
print(len(X_train))
print(len(y_train))

13120
13120


# Entrainement

In [9]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
# modèle CamemBERT pour la classification de séquences
model = CamembertForSequenceClassification.from_pretrained(model_name, num_labels=2)
#model=torch.load('modele/camembert_epoch_3.bin')
model.to(device)

model_save_path = 'modele'  # Chemin pour enregistrer le modèle
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)
    

# Optimiseur et taux d'apprentissage
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

class_weights = torch.tensor([1.0, 5.0], device=device) # adaptation des poids des classes
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

def train_epoch(model, data_loader, optimizer, device, scheduler, loss_fn):
    model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:

        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)


        model.zero_grad()

   
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)


        loss = loss_fn(outputs.logits, labels)
        
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


# Boucle d'entraînement
for epoch in range(1):
    #epoch+=1
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        scheduler,
        loss_fn
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')
    save_file_path = os.path.join(model_save_path, f'camembert_epoch_{epoch+1}.bin')
    torch.save(model, save_file_path)


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
----------


# score f1 et confusion matrix sur le test

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix
all_preds_text = []
all_labels = []
#model= torch.load('modele/camembert_epoch_1.bin')
model.eval()
all_preds_text,all_labels = prediction_model_text(model,test_loader,device,proba=False)
f1 = f1_score(all_labels, all_preds_text)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_text)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


Test F1 Score: 0.3671691792294808
Confusion Matrix:
[[ 843 1840]
 [  49  548]]
Nombre d'éléments de classe 0 détectés : 843 sur 2683
Nombre d'éléments de classe 1 détectés : 548 sur 597


# prédictions sur quelques données afin de voir ce qu'il se passe : 

In [ ]:
import torch
from transformers import CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

all_probs = []
data=X_train[273:277]
with torch.no_grad():
    for sentence in data:
        # Tokenize the sentence
        print(sentence)
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        probs = torch.softmax(logits, dim=1)
        all_probs.append(probs.cpu().numpy())



        print(outputs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


voiture il y avait un âne
SequenceClassifierOutput(loss=None, logits=tensor([[-0.2287,  0.1516]], device='cuda:0'), hidden_states=None, attentions=None)
il y avait un âne d'accord
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7145,  0.6228]], device='cuda:0'), hidden_states=None, attentions=None)
y avait un âne d'accord super
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7334,  0.6410]], device='cuda:0'), hidden_states=None, attentions=None)
avait un âne d'accord super voilà
SequenceClassifierOutput(loss=None, logits=tensor([[-0.7127,  0.6209]], device='cuda:0'), hidden_states=None, attentions=None)
